In [8]:
import pandas as pd
from matplotlib import pyplot as plt

df = pd.read_csv("D:\Thesis\BIG DATA\Database Churn\joy work\Telecom_customer churn (100000).csv")

In [9]:
num_df = df.select_dtypes(include=['float64', 'int64']).copy()
cat_df = df.select_dtypes(include=['object']).copy()

# Categorical boolean mask
categorical_feature_mask = cat_df.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = cat_df.columns[categorical_feature_mask].tolist()

import numpy as np
#conData=np.log(0.00001 + 1)
conData=0
cat_df=cat_df.fillna(conData)
num_df=num_df.fillna(conData)
cat_df=cat_df.astype(str)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_df[categorical_cols] = cat_df[categorical_cols].apply(lambda col: le.fit_transform(col))

#cat_df[categorical_cols].head(10)

change_mou=num_df['change_mou']
change_rev=num_df['change_rev']
num_df=num_df.drop(['change_mou'], axis=1)
num_df=num_df.drop(['change_rev'], axis=1)

num_df=num_df.drop(['Customer_ID'], axis=1)

churn=num_df['churn']
num_df=num_df.drop(['churn'], axis=1)


num_df=num_df.fillna(conData)

result_df = pd.concat([num_df, cat_df], axis=1)
np.nan_to_num(result_df)

result_df_op=result_df

X=result_df_op
y=churn


In [10]:
from mlencoders.weight_of_evidence_encoder import WeightOfEvidenceEncoder

enc = WeightOfEvidenceEncoder(cols=X.columns)
X_encoded = enc.fit_transform(X, y)
X_encoded.head()

,rev_Mean,mou_Mean,totmrc_Mean,da_Mean,ovrmou_Mean,ovrrev_Mean,vceovr_Mean,datovr_Mean,roam_Mean,drop_vce_Mean,...,infobase,HHstatin,dwllsize,ethnic,kid0_2,kid3_5,kid6_10,kid11_15,kid16_17,creditcd
0,0.000000,0.556517,0.017520,-0.022460,-0.015257,-0.014783,-0.033784,0.018887,0.014593,-0.011647,...,-0.025602,-0.036986,-0.045587,-0.028551,-0.002808,-0.000904,0.002464,0.00339,-0.001641,-0.021631
1,0.017520,-0.164801,0.000000,-0.022460,0.045300,-0.107643,-0.112533,0.018887,0.014593,-0.060797,...,-0.025602,-0.036986,-0.045587,-0.400201,-0.002808,-0.000904,0.002464,0.00339,-0.001641,-0.021631
2,-0.015816,0.751490,0.094113,0.047296,-0.015257,-0.014783,-0.033784,0.018887,0.014593,-0.028216,...,-0.025602,-0.036986,-0.045587,-0.028551,-0.002808,0.043076,0.002464,0.00339,-0.001641,-0.021631
3,-1.081092,0.199842,0.017520,0.047296,-0.015257,-0.014783,-0.033784,0.018887,0.014593,0.175511,...,-0.025602,-0.036986,0.052330,0.018434,0.095326,-0.000904,0.002464,0.00339,-0.001641,-0.021631
4,-0.270162,-0.339154,-0.136630,0.047296,-0.015257,-0.014783,-0.033784,0.018887,0.014593,0.063701,...,-0.025602,-0.036986,-0.037101,0.066336,-0.002808,-0.000904,0.002464,0.00339,-0.001641,-0.021631


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

In [12]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
#Create a Gaussian Classifier
classifier = GaussianNB()

classifier.fit(X_train,y_train)



y_pred = classifier.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test, y_pred))  
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
print(classification_report(y_test, y_pred))  

from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)
roc_auc

[[4943 5078]
 [1635 8344]]
pod:  0.83615592744764
pof:  0.5067358547051193
              precision    recall  f1-score   support

           0       0.75      0.49      0.60     10021
           1       0.62      0.84      0.71      9979

   micro avg       0.66      0.66      0.66     20000
   macro avg       0.69      0.66      0.65     20000
weighted avg       0.69      0.66      0.65     20000



0.6647100363712604

In [13]:
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(n_estimators=100,max_depth=2,random_state=0)
classifier.fit(X_train, y_train) 


y_pred = classifier.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test, y_pred))  
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
print(classification_report(y_test, y_pred)) 


from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)
roc_auc

[[8009 2012]
 [2955 7024]]
pod:  0.7038781441026155
pof:  0.20077836543259156
              precision    recall  f1-score   support

           0       0.73      0.80      0.76     10021
           1       0.78      0.70      0.74      9979

   micro avg       0.75      0.75      0.75     20000
   macro avg       0.75      0.75      0.75     20000
weighted avg       0.75      0.75      0.75     20000



0.7515498893350121

In [14]:
from keras.models import Sequential
from keras.layers import Dense
#create model
model = Sequential()

#get number of columns in training data
n_cols = X_train.shape[1]

#add model layers
#model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
#model.add(Dense(10, activation='relu'))
#model.add(Dense(1))

model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
model.add(Dense(250, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

#compile model using mse as a measure of model performance
#model.compile(optimizer='adam', loss='mean_squared_error')

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  

from keras.callbacks import EarlyStopping
#set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=3)
#train model
model.fit(X_train, y_train, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])
#model.fit(X_train, y_train,epochs=20, batch_size=1, verbose=1)

from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test, y_pred))  
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
print(classification_report(y_test, y_pred))  

from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)
roc_auc


Train on 64000 samples, validate on 16000 samples
Epoch 1/30
64000/64000 [==============================] - 4s 55us/step - loss: 0.3618 - acc: 0.8385 - val_loss: 0.3473 - val_acc: 0.8481
Epoch 2/30
64000/64000 [==============================] - 3s 51us/step - loss: 0.3359 - acc: 0.8532 - val_loss: 0.3326 - val_acc: 0.8524
Epoch 3/30
64000/64000 [==============================] - 3s 52us/step - loss: 0.3319 - acc: 0.8550 - val_loss: 0.3356 - val_acc: 0.8516
Epoch 4/30
64000/64000 [==============================] - 3s 50us/step - loss: 0.3297 - acc: 0.8566 - val_loss: 0.3321 - val_acc: 0.8538
Epoch 5/30
64000/64000 [==============================] - 3s 50us/step - loss: 0.3282 - acc: 0.8563 - val_loss: 0.3293 - val_acc: 0.8552
Epoch 6/30
64000/64000 [==============================] - 3s 50us/step - loss: 0.3268 - acc: 0.8574 - val_loss: 0.3309 - val_acc: 0.8552
Epoch 7/30
64000/64000 [==============================] - 3s 50us/step - loss: 0.3258 - acc: 0.8582 - val_loss: 0.3310 - val_acc

0.7515498893350121